In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from PIL import Image
import os, sys, math
import numpy as np

from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.utils import plot_model
import itertools

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

img_width, img_height = 80, 80

train_data_dir = 'drive/My Drive/Colab Notebooks/data/train/'

validation_data_dir = 'drive/My Drive/Colab Notebooks/data/validation/'
test_data_dir = 'drive/My Drive/Colab Notebooks/data/test/'
nb_train_samples = 560#stepporepoch
nb_validation_samples = 120 #validation steps
epochs = 20
batch_size = 50

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)



In [84]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

top_model_weights_path = 'CompletModelCafe3.h5'
#model.load_weights(top_model_weights_path)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

train_datagen = ImageDataGenerator( rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen  = ImageDataGenerator( rescale=1. / 255)

train_generator = train_datagen.flow_from_directory( train_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory( validation_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')

history=model.fit_generator( train_generator, steps_per_epoch=nb_train_samples // batch_size, epochs=epochs,  validation_data=validation_generator, validation_steps=nb_validation_samples // batch_size)



Found 560 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Epoch 1/20
11/11 [==============================] - 8s 719ms/step - loss: 0.8930 - acc: 0.4787 - val_loss: 0.6914 - val_acc: 0.5100
Epoch 2/20
11/11 [==============================] - 7s 664ms/step - loss: 0.6988 - acc: 0.5152 - val_loss: 0.6900 - val_acc: 0.5286
Epoch 3/20
11/11 [==============================] - 7s 655ms/step - loss: 0.6946 - acc: 0.5141 - val_loss: 0.6923 - val_acc: 0.4857
Epoch 4/20
11/11 [==============================] - 7s 656ms/step - loss: 0.7037 - acc: 0.4940 - val_loss: 0.7044 - val_acc: 0.4900
Epoch 5/20
11/11 [==============================] - 7s 663ms/step - loss: 0.6912 - acc: 0.5469 - val_loss: 0.6939 - val_acc: 0.4857
Epoch 6/20
11/11 [==============================] - 7s 656ms/step - loss: 0.7139 - acc: 0.5403 - val_loss: 0.6868 - val_acc: 0.5429
Epoch 7/20
11/11 [==============================] - 8s 707ms/step - loss: 0.6938 - acc: 0.5727 - val_loss: 0.6825 - val_acc: 0

In [85]:
def plots(ims,figsize=(80,80),rows=1,interp=False,titles=None):
  if(type(ims[0])is np.ndarray):
    ims=np.array(ims).astype(np.uint8)
    if(ims.shape[-1]!=3):
      ims=ims.transpose((0,2,3,1))
  f=plt.figure(figsize=figsize)
  cols=len(ims)//rows if len(ims)%2==0 else len(ims)//rows+1
  for i in range(len(ims)):
    sp=f.add_subplot(rows,cols,i+1)
    sp.axis('Off')
    if titles is not None:
      sp.set_title(titles[i],fontsize=16)
      
    plt.imshow(ims[i],interpolation=None if interp else 'none')
###############
test_baches=ImageDataGenerator().flow_from_directory(validation_data_dir,target_size=(80,80),classes=['class1','class2'],batch_size=10)
test_imgs,test_labels=next(test_baches)
predictions=model.predict_generator(test_baches,steps=1,verbose=0)

for i in predictions:
  print(int(i[0]))

Found 120 images belonging to 2 classes.
1
1
1
0
1
0
0
1
1
0
